### Imports and loads

In [1]:
from typing import List

import pickle as pkl
import numpy as np
import html
from pathlib import Path

from fastai.text import *
from sklearn.model_selection import train_test_split

In [11]:
DATA_PATH = Path('DATA/')

### Dataset properties, inspection, tokenization

In [12]:
DATASET_NAME = 'x_and_y_cleaned.pkl'
with open(DATA_PATH/DATASET_NAME, 'rb') as f:
    articles, categories = pkl.load(f)

In [55]:
CLASSES = set(categories)
ARTICLE_COUNT = len(articles)
CLASS_COUNT = len(CLASSES)
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag
MAX_SIZE = 250

print(ARTICLE_COUNT)
print(CLASS_COUNT)

48514
138


In [56]:
# Dataset examples:
index = 0
print('ARTICLE: ', articles[index][0:110], '...')
print('CATEGORY: ', categories[index])

ARTICLE:  Kas parima aastavahetuse programmi pani eetrisse ETV, Kanal 2 või hoopis TV3? ETVst näegid vaatajad saateid "V ...
CATEGORY:  televeeb/tvuudised


In [54]:
# Get median/average word count
print(np.median([len(x.split(' ')) for x in articles]))
print(np.mean([len(x.split(' ')) for x in articles]))

261.0
387.0457805994146


In [85]:
# No enter key version
articles_pad_tag = [' '.join([BOS] + x.split(' ')[:MAX_SIZE - 2] + [FLD]) if len(x.split(' ')) > MAX_SIZE else ' '.join([BOS] + x.split(' ') + [FLD] + ['0' for i in range(MAX_SIZE - 2 - len(x.split(' ')))]) for x in articles]

# articles_pad_tag = []
# for x in articles:
#     xs = x.split(' ')
#     if len(xs) > MAX_SIZE:
#         articles_pad_tag.append(' '.join([BOS] + xs[:MAX_SIZE - 2] + [FLD]))
#     else:
#         articles_pad_tag.append(' '.join([BOS ] + xs + [FLD] + ['0' for i in range(MAX_SIZE - 2 - len(xs))]))
                            
print(len(articles_pad_tag[0].split(' ')))
print(articles_pad_tag[0])

print(len(articles_pad_tag[5].split(' ')))
print(articles_pad_tag[5])

#articles_tagged = [BOS + ' ' + x + ' ' + FLD for x in articles]

250
xbos Kas parima aastavahetuse programmi pani eetrisse ETV, Kanal 2 või hoopis TV3? ETVst näegid vaatajad saateid "Võrno ja Oja aastavahetuse pidu", "Rahvasaadik Pius", "Ivan Orav", "Edekabel 2014" ja "Rahva Tujurikkuja". Kanal 2 panustas saadetele "Unustamatu 2014", "Eetriapsud 2014", "Reporteri aastalõpp", "Naabriplika aastalõputrall" ja "Aastavahetus Kanal 2ga". TV3 eetris oli aga "Me armastame Eestit", "Mida toob aasta 2015 ", "Padjaklubi näärid", "Mardi ja Jani parimad näod" ning "Lauri ja Uku parimad näod". Millise kanali valik on või lõpuks oli sinu arvates parim? Milline sketš meeldis kõige rohkem? Mida tahad laita? Kirjuta! xfld 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
250
xbos TV3 aastalõpusaates "Mida 

In [86]:
np.random.seed(42)
train_texts, val_texts, train_labels, val_labels = train_test_split(articles_pad_tag, categories, test_size=0.1, random_state=42)
pickle.dump([train_texts, val_texts, train_labels, val_labels], open(DATA_PATH/'tokens'/'trnx_valx_trny_valy_split.pkl', 'wb'))

### Tokenize

In [87]:
max_vocab = 60000
min_freq = 5

In [88]:
tok_train = Tokenizer(lang='xx').proc_all_mp(partition_by_cores(train_texts))
tok_val = Tokenizer(lang='xx').proc_all_mp(partition_by_cores(val_texts))

In [89]:
freq = Counter(p for o in tok_train for p in o)
print(len(tok_train))
freq.most_common(25)

43662


[(',', 657926),
 ('.', 559252),
 ('"', 217175),
 ('ja', 210514),
 ('on', 197759),
 ('et', 150766),
 ('ei', 106727),
 ('kui', 74991),
 ('ta', 66639),
 ('ka', 58212),
 ('oli', 51101),
 ('oma', 46727),
 ('-', 46020),
 ('ning', 45314),
 ('see', 45285),
 ('xbos', 43662),
 ('xfld', 43662),
 ('0', 42597),
 ('aga', 38936),
 ('t_up', 31812),
 ('mis', 31436),
 ('ma', 30478),
 ('siis', 29830),
 ('kes', 29218),
 ('tema', 28739)]

In [90]:
print(tok_val[5])

['xbos', 'vehklemisliidu', 'president', ',', 'riigikogu', 'liige', 'margus', 'hanson', 'tõdes', ',', 'et', 'naiskond', 'vehkles', 'kaunilt', 'kuni', 'finaalini', '.', '"', 'naised', 'olid', 'väga', 'tublid', '.', 'meil', 'on', 'noor', ',', 'perspektiivikas', 'ja', 'arenev', 'võistkond', ':', 'teise', 'kohaga', 'tuleb', 'igati', 'rahul', 'olla', ',', 'sest', 'ega', 'jõu', 'ja', 'võimu', 'vastu', 'ei', 'saa', '!', '"', 'hanson', 'lisas', ',', 'et', 'teda', 'rõõmustab', 'sten', 'priinitsa', 'individuaalturniiril', 'saadud', 'kaheksas', 'koht', ',', 'millega', 'mees', 'suurendab', 'ka', 't_up', 'eok', 'toetusraha', '.', '"', 'meie', 'vehklejad', 'on', 'tõestanud', ',', 'et', 'neid', 'saab', 'usaldada', '.', 'sportlased', 'seavad', 'kõrged', 'sihid', 'ja', 'on', 'võimelised', 'neid', 'täitma', ';', '"', 'kinnitas', 'ta', '.', 'ühtlasi', 'märkis', 'hanson', ',', 'et', 'suur', 'on', 'treener', 'igor', 'tšikinjovi', 'panus', '.', '"', 'ta', 'on', 'toonud', 'värsket', 'verd', 'ja', 'hingamist',

In [91]:
freq_val = Counter(p for o in tok_val for p in o)
print(len(tok_val))
freq_val.most_common(25)

4852


[(',', 72534),
 ('.', 62293),
 ('"', 23741),
 ('ja', 23599),
 ('on', 21847),
 ('et', 16600),
 ('ei', 11625),
 ('kui', 8402),
 ('ta', 7294),
 ('ka', 6594),
 ('oli', 5541),
 ('ning', 5219),
 ('oma', 5142),
 ('-', 5101),
 ('see', 4893),
 ('xbos', 4852),
 ('xfld', 4852),
 ('0', 4743),
 ('aga', 4345),
 ('mis', 3589),
 ('t_up', 3475),
 ('ma', 3323),
 ('tema', 3264),
 ('eesti', 3248),
 ('siis', 3235)]

In [92]:
np.save(DATA_PATH/'tokens/tok_train_pad.npy', tok_train)
np.save(DATA_PATH/'tokens/tok_val_pad.npy', tok_val)

In [93]:
itos = [o for o,c in freq.most_common(max_vocab) if c>min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

In [94]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

In [95]:
train_lm = np.array([[stoi[o] for o in p] for p in tok_train])
val_lm = np.array([[stoi[o] for o in p] for p in tok_val])

In [96]:
np.save(DATA_PATH/'tokens'/'trn_ids_pad.npy', train_lm)
np.save(DATA_PATH/'tokens'/'val_ids_pad.npy', val_lm)
pickle.dump(itos, open(DATA_PATH/'tokens'/'itos_pad.pkl', 'wb'))

### Load tokenized data

In [16]:
train_texts, val_texts, train_labels, val_labels = pickle.load(open(DATA_PATH/'tokens'/'trnx_valx_trny_valy_split.pkl', 'rb'))
train_lm = np.load(DATA_PATH/'tokens'/'trn_ids.npy')
val_lm = np.load(DATA_PATH/'tokens'/'val_ids.npy')
itos = pickle.load(open(DATA_PATH/'tokens'/'itos.pkl', 'rb'))

In [31]:
print(train_texts[0])

xbos Peaminister Taavi Rõivas jätab võimutüli tõttu ära visiidid Leedusse ja Rootsi, teda asendab väliskaubandus- ja ettevõtlusminister Anne Sulling.  Valitsuse pressiesindaja kinnitas pühapäeva pärastlõunal, et Rõivas ei sõida esmaspäeval visiidile Leetu ja Rootsi. Pressiesindaja teatel jäävad visiidid ära "seoses ametikohustustega Eestis". Reformierakonna esimees, peaminister Taavi Rõivas pidi esmaspäeval koos teiste Balti riikide valitsusjuhtidega osalema Leedus Klaipedas aset leidval LNG ujuvterminali saabumistseremoonial. Enne tseremooniat pidi aset leidma peaministrite ning Ameerika Ühendriikide esindajate ühine töölõuna. Pärastlõunal pidi Rõivas suunduma edasi Stockholmi, kus toimub Balti- ja Põhjamaade tippkohtumine. Rootsi, Soome, Norra, Islandi, Taani, Eesti, Läti ja Leedu peaministrite kohtumisel räägitakse majanduse olukorrast Euroopas, transatlantilistest suhetest ning Ukrainaga seotud arengutest. Pühapäeval kohtuvad Reformierakonna ja Sotsiaaldemokraatliku Erakonna esimeh

In [32]:
print(train_lm[0])

[34, 659, 789, 880, 1415, 0, 275, 54, 56898, 0, 5, 771, 2, 91, 38930, 0, 5, 59121, 1607, 35257, 3, 111, 1021, 599, 563, 3011, 2276, 2, 7, 880, 8, 6322, 875, 8155, 20625, 5, 771, 3, 599, 1104, 827, 56898, 54, 4, 722, 0, 139, 4, 3, 1332, 1075, 2, 659, 789, 880, 387, 875, 88, 355, 804, 1432, 0, 4156, 5188, 0, 2835, 0, 32, 13676, 0, 0, 3, 113, 0, 387, 2835, 4588, 41646, 16, 709, 2361, 10782, 3513, 0, 3, 2276, 387, 880, 0, 160, 5461, 2, 46, 580, 0, 5, 13848, 18935, 3, 771, 2, 445, 2, 1279, 2, 8513, 2, 2720, 2, 30, 2, 874, 5, 1577, 41646, 3096, 2368, 3858, 2806, 1231, 2, 0, 6323, 16, 10509, 401, 27516, 3, 1082, 9917, 1332, 5, 9358, 1370, 0, 2, 7, 3505, 2630, 1271, 240, 0, 10510, 1926, 3, 35]


In [34]:
print(train_labels[0])

uudised/eesti


### Feed-forward NN

In [2]:
torch.__version__

'0.3.1.post2'

In [103]:
class SimpleFNN(nn.Module):
    def __init__(self, vocab_size, num_outputs, num_l, neurons: List[int], e_size=200):
        super(SimpleFNN, self).__init__()
        self.e = nn.Embedding(vocab_size, e_size)
        self.input_l = nn.Linear(e_size, neurons[0])
        self.middle_l = nn.ModuleList()
        for i in range(num_l):
            self.middle_l.append(nn.Linear(neurons[i], neurons[i+1]))
        self.output_l = nn.Linear(neurons[-1], num_outputs)
        
    def forward(self, x):
        x = F.relu(self.e(x))
        x = F.relu(self.input_l(x))
        for l in self.middle_l:
            x = F.relu(l(x))
        return F.softmax(self.output_l(x))
        

In [104]:
fnn = SimpleFNN(max_vocab, CLASS_COUNT, 4, [200, 300, 100, 50, 20])
print(fnn)

SimpleFNN(
  (e): Embedding(60000, 200)
  (input_l): Linear(in_features=200, out_features=200, bias=True)
  (middle_l): ModuleList(
    (0): Linear(in_features=200, out_features=300, bias=True)
    (1): Linear(in_features=300, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=20, bias=True)
  )
  (output_l): Linear(in_features=20, out_features=138, bias=True)
)


In [105]:
# Needs another dim and probably one hot encoding
#fnn(Variable(torch.Tensor(train_lm[0])))

TypeError: torch.index_select received an invalid combination of arguments - got ([32;1mtorch.FloatTensor[0m, [32;1mint[0m, [31;1mtorch.FloatTensor[0m), but expected (torch.FloatTensor source, int dim, torch.LongTensor index)